In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
import os
import numpy as np
from operator import itemgetter 
from random import shuffle 
import optuna
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

In [ ]:
files = []
classes = ['kiss', 'handshake', 'hug', 'highfive']

for class_n, c in enumerate(classes):
    f = os.listdir('output/'+c)
    f = list(map(lambda x: 'output/'+c+'/'+x, f))
    
    filtered = list(filter(lambda x: x.split('.')[-1] == 'npy', f))
    
    files.extend(filtered)

In [ ]:
def convertClass(classname):
    if classname == None:
        raise Error('No classname!')
        
    mapping = {key: value for (value, key) in enumerate(classes)}
    
    return mapping[classname]

def onehot(y):
    onehot = np.zeros((4))
    onehot[y] = 1
    return tf.constant(onehot)

def batcherize_train():
    indices = train_idx

    for _ in range(epochs):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield np.array([df]), np.array([class_onehot])

def batcherize_test():
    indices = test_idx
        
    for _ in range(epochs+1):
        for idx in indices:
            df = np.load(files[idx])/256
            class_name = files[idx].split('/')[-2]
            class_int = convertClass(class_name)
            class_onehot = onehot(class_int)

            yield np.array([df]), np.array([class_onehot])

In [ ]:
epochs = 60
test_size = 0.25

shuffled_idx = np.random.permutation(len(files))
split_at = int(len(shuffled_idx) * (1-test_size))

train_idx = shuffled_idx[:split_at]
test_idx = shuffled_idx[split_at:]

train_size = len(train_idx)
test_size = len(test_idx)

In [ ]:
model = Sequential()

model.add(InputLayer((None, 128, 128,3)))
model.add(TimeDistributed(
    Conv2D(
        filters=4, 
        kernel_size=4, 
        data_format='channels_last', 
        padding="same",
        kernel_initializer='he_normal',
    ), 
    name='Conv2D'))
model.add(TimeDistributed(Flatten(), name='Flatten'))
model.add(LSTM(64, dropout=0.3))
model.add(Dense(512,kernel_regularizer=tf.keras.regularizers.L2(0.006)))
model.add(Dropout(0.45))
model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001), 
    loss=tf.keras.losses.CategoricalCrossentropy(), 
    metrics=['accuracy']
)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * tf.math.exp(-0.1)
scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

h = model.fit(
    batcherize_train(),
    epochs=epochs, 
    steps_per_epoch=train_size,
    validation_data=batcherize_test(),
    validation_steps=test_size,
)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(16,4))

ax[0].plot(h.history['loss'], label='Training')
ax[0].plot(h.history['val_loss'], label='Test')
ax[0].set_title('Loss')
ax[0].legend()

ax[1].plot(h.history['accuracy'], label='Training')
ax[1].plot(h.history['val_accuracy'], label='Test')
ax[1].set_title('Accuracy')
ax[1].legend()

In [ ]:
y_pred = model.predict(batcherize_test(), steps=test_size)

y_true = []

for idx in test_idx:
    class_name = files[idx].split('/')[-2]
    class_int = convertClass(class_name)
    class_onehot = onehot(class_int)

    y_true.append(np.array([class_onehot]))
    
y_true = np.array(y_true)
y_true = y_true.reshape(200, 4)
y_true = np.argmax(y_true, axis=1)

y_pred = np.argmax(y_pred, axis=1)

In [ ]:
conf_m = sns.heatmap(tf.math.confusion_matrix(y_true, y_pred), annot=True, xticklabels=classes, yticklabels=classes)
conf_m.set(xlabel='Predicted labels', ylabel='True labels')

conf_m.figure.savefig('./graphs/cnn/cnn_confusion_matrix.png', dpi=150, bbox_inches = "tight")